In [ ]:
!pip install soundata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.0/162.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 27.0 MB/s eta 0:00:00


In [ ]:
import soundata

dataset = soundata.initialize('urbansound8k')
dataset.download()  # download the dataset
dataset.validate()  # validate that all the expected files are there

print("Total de clips:", len(dataset.clip_ids))


5.61GB [07:16, 13.8MB/s]                            
1.15MB [00:01, 649kB/s]                            
100%|██████████| 8732/8732 [00:42<00:00, 204.14it/s]

Total de clips: 8732


In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models



In [ ]:
SR = 22050
DURATION = 4.0
N_MFCC = 40

def load_clip_mfcc(clip):
    y = clip.audio
    sr = clip.sample_rate

    # Resample
    if sr != SR:
        y = librosa.resample(y, orig_sr=sr, target_sr=SR)

    # Asegurar duración fija
    max_len = int(SR * DURATION)
    if len(y) < max_len:
        y = np.pad(y, (0, max_len - len(y)))
    else:
        y = y[:max_len]

    # Calcular MFCC
    mfcc = librosa.feature.mfcc(y=y, sr=SR, n_mfcc=N_MFCC)
    return mfcc  # shape: (40, T)


In [ ]:
def load_data_for_fold(dataset, test_fold):
    X_train, y_train = [], []
    X_test, y_test = [], []

    for cid in dataset.clip_ids:
        clip = dataset.clip(cid)
        mfcc = load_clip_mfcc(clip)
        label = clip.class_id
        fold = clip.fold

        if fold == test_fold:
            X_test.append(mfcc)
            y_test.append(label)
        else:
            X_train.append(mfcc)
            y_train.append(label)

    return np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)


In [ ]:
def prepare_lstm_features(X):
    X_out = []
    for mfcc in X:
        X_out.append(mfcc.T)  # (40, T) → (T, 40)
    return np.array(X_out)


In [ ]:
def build_lstm_model(n_timesteps, n_features):
    model = models.Sequential([
        layers.Input(shape=(n_timesteps, n_features)),
        layers.LSTM(64, return_sequences=True),
        layers.LSTM(32),
        layers.Dense(32, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [ ]:
accuracies = []

print("\n===================================================")
print("🔵 INICIANDO 10-FOLD CROSS VALIDATION (LSTM)")
print("===================================================\n")

for fold in range(1, 11):

    print(f"\n===================================================")
    print(f"🚀 Ejecutando Fold {fold} ...")
    print("===================================================\n")

    # 1. Cargar datos del fold
    X_train_raw, y_train, X_test_raw, y_test = load_data_for_fold(dataset, test_fold=fold)

    print(f"Fold {fold}: Train={len(X_train_raw)}, Test={len(X_test_raw)}")
    print(f"Shapes MFCC: train={X_train_raw.shape}, test={X_test_raw.shape}")

    # 2. Preparar MFCC → LSTM (T, 40)
    X_train_lstm = prepare_lstm_features(X_train_raw)
    X_test_lstm  = prepare_lstm_features(X_test_raw)

    n_timesteps = X_train_lstm.shape[1]
    n_features  = X_train_lstm.shape[2]

    print(f"Shapes LSTM: train={X_train_lstm.shape}, test={X_test_lstm.shape}")

    # 3. Crear modelo LSTM
    lstm_model = build_lstm_model(n_timesteps, n_features)

    # 4. Entrenar
    history = lstm_model.fit(
        X_train_lstm, y_train,
        epochs=20,
        batch_size=32,
        validation_data=(X_test_lstm, y_test),
        verbose=1
    )

    # 5. Evaluar
    test_loss, test_acc = lstm_model.evaluate(X_test_lstm, y_test, verbose=0)
    accuracies.append(test_acc)

    print(f"\n🎯 Accuracy del Fold {fold}: {test_acc:.4f}")
    print("---------------------------------------------------")

# ---------------------------------------------------
# RESULTADOS FINALES
# ---------------------------------------------------
import numpy as np

print("\n===================================================")
print("🔵 RESULTADOS DEL 10-FOLD CROSS VALIDATION (LSTM)")
print("===================================================\n")

print("Accuracies por fold:", accuracies)

mean_acc = np.mean(accuracies)
std_acc  = np.std(accuracies)

print(f"\n📌 Accuracy promedio:      {mean_acc:.4f}")
print(f"📌 Desviación estándar:    {std_acc:.4f}")

print("\n===================================================")
print("🏁 ENTRENAMIENTO COMPLETO")
print("===================================================")



🔵 INICIANDO 10-FOLD CROSS VALIDATION (LSTM)


🚀 Ejecutando Fold 1 ...



NameError: name 'load_data_for_fold' is not defined